In [ ]:
!pip install datasets
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.3 MB/s eta 0:00:00


In [ ]:
import sklearn

In [ ]:
print(sklearn.__version__)

1.2.2


In [ ]:
# !pip install scikit-learn==1.3.0

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from datasets import load_dataset

from google.colab import data_table

data_table.disable_dataframe_formatter()
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import cross_val_score, KFold

import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DataPath = "/content/drive/MyDrive/Magang/ThreatDetection/Data/"

In [ ]:
df_concat = pd.read_csv(f"{DataPath}concanate.csv")
df_concat.drop(columns = "Unnamed: 0", inplace= True)

In [ ]:
df_ccnt_cleaned = pd.read_csv(f"{DataPath}concanite_cleaned2.csv")
df_ccnt_cleaned.drop(columns = "Unnamed: 0", inplace= True)
df_ccnt_cleaned.columns = ['text']

In [ ]:
df_clean= pd.concat([df_ccnt_cleaned, df_concat.iloc[:, 1:]], axis=1)

In [ ]:
df_clean.head()

,text,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,sexual_orientation,disability,national_origin,violence
0,saat cowok usaha lacak perhati loe lantas reme...,1,1,0,0,0,0,0,1,0,0,0,0
1,telat ngasih sarap gaul cigax jifla cal noh licew,1,0,0,0,0,0,0,0,0,0,0,0
2,kadang percaya tuhan jatuh kadang tuhan ningga...,0,0,0,0,0,0,0,0,0,0,0,0
3,mata sipit liat,0,0,0,0,0,0,0,0,0,0,0,0
4,kaum cebong kapir udah liat dongok dongok,1,0,1,1,0,0,0,0,0,0,0,0


In [ ]:
df_X = df_concat.iloc[:,0]
df_y = df_concat.iloc[:,1:]
df_y = df_y.reset_index()
df_y.drop(columns = 'index', inplace = True)
df_cleaned = pd.concat([df_ccnt_cleaned,df_y], axis=1)

df_cleaned.dropna(inplace=True)

In [ ]:
df_cleaned.shape

(13412, 13)

In [ ]:
with open(f'{DataPath}/Model/fitted_vectorizer.pkl', 'rb') as file:
    vectorizer = pickle.load(file)
x_trans = vectorizer.transform(df_cleaned.iloc[:,0].values)
feature_names = vectorizer.get_feature_names_out()
array = x_trans.toarray()
arraylist = array.tolist()

# dfTFIDF = pd.DataFrame(arraylist, columns=feature_names)
# dfTFIDF.head()

In [ ]:
X = array

In [ ]:
Y = df_cleaned.iloc[:,1:].values

In [ ]:
Y.shape

(13412, 12)

In [ ]:
X.shape

(13412, 1705)

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()
y = df_clean.iloc[:,1:].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
# y_train = mlb.fit_transform(y_train)
# y_test = mlb.transform(y_test)

In [ ]:
X_train.shape

(12070, 1705)

In [ ]:
y_train.shape

(12070, 12)

In [ ]:
X_test.shape

(1342, 1705)

In [ ]:
X_test[0].shape

(1705,)

In [ ]:
from tensorflow import keras

In [ ]:
X_train.shape[1]

1705

In [ ]:
y_train.shape

(12070, 12)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(200, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(y_train.shape[1], activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
378/378 [==============================] - 5s 10ms/step - loss: 0.2549 - accuracy: 0.5879
Epoch 2/100
378/378 [==============================] - 3s 7ms/step - loss: 0.1605 - accuracy: 0.5169
Epoch 3/100
378/378 [==============================] - 3s 7ms/step - loss: 0.1388 - accuracy: 0.5052
Epoch 4/100
378/378 [==============================] - 3s 7ms/step - loss: 0.1258 - accuracy: 0.5037
Epoch 5/100
378/378 [==============================] - 4s 10ms/step - loss: 0.1149 - accuracy: 0.5125
Epoch 6/100
378/378 [==============================] - 3s 9ms/step - loss: 0.1041 - accuracy: 0.5069
Epoch 7/100
378/378 [==============================] - 3s 7ms/step - loss: 0.0934 - accuracy: 0.5068
Epoch 8/100
378/378 [==============================] - 3s 7ms/step - loss: 0.0821 - accuracy: 0.5175
Epoch 9/100
378/378 [==============================] - 2s 6ms/step - loss: 0.0707 - accuracy: 0.5201
Epoch 10/100
378/378 [==============================] - 3s 9ms/step - loss: 0.0604 - accu

In [ ]:
y_pred = model.predict(X_test[1].reshape(1,X_test.shape[1]))

1/1 [==============================] - 0s 66ms/step


In [ ]:
y_pred

array([[2.7355134e-07, 9.9999601e-01, 2.3860736e-09, 2.7818782e-08,
        1.1815654e-10, 9.4921424e-17, 3.1914414e-08, 9.9995208e-01,
        3.9929523e-19, 4.8638153e-19, 8.0910126e-18, 3.0303778e-17]],
      dtype=float32)

In [ ]:
y_pred[0]

array([2.7355134e-07, 9.9999601e-01, 2.3860736e-09, 2.7818782e-08,
       1.1815654e-10, 9.4921424e-17, 3.1914414e-08, 9.9995208e-01,
       3.9929523e-19, 4.8638153e-19, 8.0910126e-18, 3.0303778e-17],
      dtype=float32)

In [ ]:
np.argsort(y_pred[0])[::-1]

In [ ]:
index_label = np.argsort(y_pred[0])[::-1]

In [ ]:
index_label

In [ ]:
prob_label = y_pred[0][index_label] * 100

In [ ]:
prob_label

array([9.9999603e+01, 9.9995209e+01, 2.7355134e-05, 3.1914415e-06,
       2.7818783e-06, 2.3860736e-07, 1.1815654e-08, 9.4921426e-15,
       3.0303779e-15, 8.0910128e-16, 4.8638154e-17, 3.9929524e-17],
      dtype=float32)

In [ ]:
print(y_test[1].tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
X_test[0].shape

(1705,)

In [ ]:
X_test.shape

(2683, 1705)

In [ ]:
X_test[0].shape

(1705,)

In [ ]:
y_pred

array([[0.99959123, 0.5731743 ],
       [0.9999255 , 0.9827179 ],
       [0.9999854 , 0.06060532],
       ...,
       [0.9997492 , 0.9267743 ],
       [0.9999951 , 0.00116853],
       [0.99997276, 0.99988025]], dtype=float32)

In [ ]:
y_test

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 1]])

In [ ]:
model_log = MultiOutputClassifier(LogisticRegression())

In [ ]:
model_log.fit(X_train, y_train)

MultiOutputClassifier(estimator=LogisticRegression())

In [ ]:
y_pred = model_log.predict(X_test)

In [ ]:
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
df_cleaned.head()

In [ ]:
y_test.shape

(2683, 12)

In [ ]:
df_cleaned.iloc[:,1:].columns

Index(['Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race',
       'HS_Physical', 'HS_Gender', 'HS_Other', 'sexual_orientation',
       'disability', 'national_origin', 'violence'],
      dtype='object')

In [ ]:
df_cleaned.iloc[:,1:].shape

(13412, 12)

In [ ]:
def clf_report(y_test, y_predict):
  for i in range(11):
    print("\n",df_cleaned.iloc[:,1:].columns[i+1])
    cm_y1_forest = classification_report(y_test[:,i], y_predict[:,i])
    print(cm_y1_forest)
    print("->"*50)

In [ ]:
clf_report(y_test, y_pred)

In [ ]:

label = ['Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race','HS_Physical', 'HS_Gender', 'HS_Other', 'sexual_orientation','disability', 'national_origin', 'violence']

In [ ]:
label

['Abusive',
 'HS_Individual',
 'HS_Group',
 'HS_Religion',
 'HS_Race',
 'HS_Physical',
 'HS_Gender',
 'HS_Other',
 'sexual_orientation',
 'disability',
 'national_origin',
 'violence']

In [ ]:
y_test[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
np.where(y_test[10])[0].tolist()

[0, 1, 7]

In [ ]:
y_pred[15]

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [ ]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
# import nltk
import pickle

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# from google.colab import data_table

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pathlib import Path


# DataPath = DataPath = str(Path(__file__).parent.absolute())

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
if not np.where(y_pred[1])[0].tolist():
  print('Netral')
else:
  for i in np.where(y_pred[1])[0]:
    print(label[i])

Netral


In [ ]:
def preprocess(sentence:list):
    # case Folding
    sentences = [sentence.lower() for sentence in sentence]
    # print(sentences)


    # tokenizer
    tokenizer     = RegexpTokenizer(r'(?<!\S)[A-Za-z]+(?!\S)|(?<!\S)[A-Za-z]+(?=:(?!\S))')
    # hasil_token   = tokenizer.tokenize(sentences[1])
    token_sentence = [tokenizer.tokenize(sentence) for sentence in sentences]
    # print(token_sentence)


    # ah, al, ala, an, and, as, atau, awal, be, ber, bin, bro, bu, bung, but, by, cak, cc, dan, deh, dengan, di, dil, dong, dua, eh, fadli, fahri
    # fan, for, front, full, harto, hasan, he, hehe, hem, ia, ih, in, is, itu, iya, je, juga, kah, kak, kan, kang, kat, "kau", ke, km, kok, ku, ky
    # lah, like, lima, live, "lo", me, mei, men, min, mk, mu, my, nah, nak, namun, nek, nih, no, nya, of, oh, oi, oke, -pada, pak, per, ppp, pun, ra
    # rang, rt, sa, se, seba, si, sing, ta, tang, tar, tau, the, ting, to, toh, tomohon, uniform, up, untuk, user, wah, wkwk, ya, yaitu, yakni, yang, zon

    # yuk,yg,yaa,wkwkwk,tdk,so,kyk,ko,jg,jgn,jd,jdi,hrs,haha,gw,gk,ga,ente,emg,elo,dr,dlm,dgn,byk,bkn,aq,ama,aku,adlh,ad

    # ad,adlh,aku,ama,aq,bgt,bkn,blg,bnyk,bs,byk,da,dah,dapet,dapat,dengar,dg,dgn,dll,ga,gmn,gt,gw,haha,hahaha,jd,jdi,jgn,ka,karna,ken,kl,klo,kyk,lg,lho,lu,mmg,mo,msh,ngga,nggak,ntar,yg,yo
    words = """ad,adlh,aku,ama,aq,bgt,bkn,blg,bnyk,bs,byk,da,dah,dapet,dapat,dengar,dg,dgn,dll,ga,gmn,gt,gw,haha,hahaha,jd,jdi,jgn,ka,karna,ken,kl,klo,kyk,lg,lho,lu,mmg,mo,msh,ngga,nggak,ntar,yg,yo,yuk,yg,yaa,wkwkwk,tdk,so,kyk,ko,jg,jgn,jd,jdi,hrs,haha,gw,gk,ga,ente,emg,elo,dr,dlm,dgn,byk,bkn,aq,ama,aku,adlh,ad,gue,aja,aku,pengguna,ah,al,ala,an,and,as,atau,awal,be,ber,bin,bro,bu,bung,but,by,cak,cc,dan,deh,dengan,di,dil,dong,dua,eh,fadli,fahri,fan,for,front,full,harto,hasan,he,hehe,hem,ia,ih,in,is,itu,iya,je,juga,kah,kak,kan,kang,kat,kau,ke,km,kok,ku,ky,lah,like,lima,live,lo,me,mei,men,min,mk,mu,my,nah,nak,namun,nek,nih,no,nya,of,oh,oi,oke,pak,per,ppp,pun,ra,rang,rt,sa,se,seba,si,sing,ta,tang,tar,ter,tau,the,ting,to,toh,tomohon,uniform,up,untuk,user,wah,wkwk,ya,yaitu,yakni,yang,zon"""
    word = words.split(",")
    # word = np.reshape(word, -1)
    # word.tolist()
    # stopword
    wordlist= set(stopwords.words('indonesian') + word)
    sw_sentences = [[word for word in sentence if word not in wordlist]for sentence in token_sentence]
    cleaned_sentences = [' '.join(sentence) for sentence in sw_sentences]
    # print(cleaned_sentences)


    #[str(sentence) for sentence in sw_sentences]
    # stemming
    stemmer     = StemmerFactory().create_stemmer()
    # input_stemm   = str(sw_sentences)
    # hasil_stemm   = stemmer.stem(input_stemm)
    # print (hasil_stemm)
    stem_sentence = [stemmer.stem(sentence) for sentence in cleaned_sentences]
    # print(stem_sentence)
    return stem_sentence

label = ['Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race','HS_Physical', 'HS_Gender', 'HS_Other', 'sexual_orientation','disability', 'national_origin', 'violence']


In [ ]:
import pickle

In [ ]:
# pickle.dump(model_log, open('logistic_model.pkl', 'wb'))

In [ ]:

loaded_model = pickle.load(open('logistic_model.pkl', 'rb'))

In [ ]:
vectorize = pickle.load(open('fitted_vectorizer.pkl', 'rb'))

In [ ]:
kalimat = preprocess(["Dasar China"])

In [ ]:
kalimat_vector = vectorize.transform(kalimat)

In [ ]:
y_pred = loaded_model.predict(kalimat_vector.toarray())

In [ ]:
y_pred[0].tolist()

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
label = ['Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race','HS_Physical', 'HS_Gender', 'HS_Other', 'sexual_orientation','disability', 'national_origin', 'violence']


In [ ]:
np.where(y_pred[0])

(array([0]),)

In [ ]:
np.where(y_pred[0].tolist())

(array([0]),)

In [ ]:
classification = []

if not np.where(y_pred[0].tolist()):
  print("Gagal")
  classification.append(['Netral'])
else:
  print("Berhasil")
  for i in np.where(y_pred)[0]:
      classification.append(label[i])

Berhasil


In [ ]:
classification

[]

In [ ]:
classification.append('Netral')

In [ ]:
classification

[['Netral'], 'Netral']

# Ini Kebawah merupakan codingan lama

In [ ]:
jumlahNilaiTiapColumn = df_concat.iloc[:, 1:].sum()
jumlahNilaiTiapColumn.head()

Abusive          5043
HS_Individual    3710
HS_Group         2283
HS_Religion       873
HS_Race           642
dtype: int64

In [ ]:
fig = px.bar(data_frame= jumlahNilaiTiapColumn, x = jumlahNilaiTiapColumn.index, y = jumlahNilaiTiapColumn.values,
             title = "jumlah nilai setiap column", labels = {'y' : "jumlah", 'index' : "nama column"})
fig.update_xaxes(tickangle=90)
fig.show()

In [ ]:
px.bar(data_frame= df_concat.HS_Physical.value_counts(), x = df_concat.HS_Physical.value_counts().index, y = df_concat.HS_Physical.value_counts().values)

In [ ]:
df_X = df_concat.iloc[:,0]
df_y = df_concat.iloc[:,1:]
df_y = df_y.reset_index()
df_y.drop(columns = 'index', inplace = True)
df_cleaned = pd.concat([df_ccnt_cleaned,df_y], axis=1)

df_cleaned.dropna(inplace=True)

In [ ]:
df_y = df_y.reset_index()
df_y.drop(columns = 'index', inplace = True)

In [ ]:
df_cleaned = pd.concat([df_ccnt_cleaned,df_y], axis=1)


In [ ]:
df_cleaned.dropna(inplace=True)

In [ ]:
with open(f'{DataPath}fitted_vectorizer.pkl', 'rb') as file:
    vectorizer = pickle.load(file)
x_trans = vectorizer.transform(df_cleaned.iloc[:,0].values)
feature_names = vectorizer.get_feature_names_out()
array = x_trans.toarray()
arraylist = array.tolist()

dfTFIDF = pd.DataFrame(arraylist, columns=feature_names)
dfTFIDF.head()

,abang,abis,abu,acara,ada,adab,adat,ade,adek,adem,...,yah,yahudi,yakin,yaudah,ye,you,zalim,zaman,zionis,zonk
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X = dfTFIDF
y = df_cleaned.iloc[:,1:]

In [ ]:
X.shape

(13412, 1705)

In [ ]:
y = y.reset_index()
y.drop(columns = 'index', inplace= True)

In [ ]:
y.shape

(13412, 12)

In [ ]:
df = pd.concat([X, y], axis = 1)

In [ ]:
sample_dict = { 'S1': [10, 20, np.NaN, np.NaN],
                'S2': [5, np.NaN, np.NaN, 29],
                'S3': [15, 20, np.NaN, np.NaN],
                'S4': [21, 22, 23, 25],
                'Subjects': ['Hist', 'Finan', 'Maths', 'Geog']}


In [ ]:
# Create a DataFrame from dictionary
df = pd.DataFrame(sample_dict)
# Set column 'Subjects' as Index of DataFrame
df = df.set_index('Subjects')

In [ ]:
df

,S1,S2,S3,S4
Subjects,,,,
Hist,10.0,5.0,15.0,21
Finan,20.0,NaN,20.0,22
Maths,NaN,NaN,NaN,23
Geog,NaN,29.0,NaN,25


In [ ]:
df.applymap(lambda l: l if not np.isnan(l) else np.random.choice([0, 1]))

,S1,S2,S3,S4
Subjects,,,,
Hist,10.0,5.0,15.0,21
Finan,20.0,0.0,20.0,22
Maths,0.0,1.0,1.0,23
Geog,1.0,29.0,1.0,25


In [ ]:
df.head()

,S1,S2,S3,S4
Subjects,,,,
Hist,10.0,5.0,15.0,21
Finan,20.0,NaN,20.0,22
Maths,NaN,NaN,NaN,23
Geog,NaN,29.0,NaN,25


In [ ]:
y.columns

Index(['Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race',
       'HS_Physical', 'HS_Gender', 'HS_Other', 'sexual_orientation',
       'disability', 'national_origin', 'violence'],
      dtype='object')

In [ ]:
dfAbusive = pd.concat([X, y['Abusive']], axis = 1)
dfHS_Individual = pd.concat([X, y['HS_Individual']], axis = 1)
dfHS_Group = pd.concat([X, y['HS_Group']], axis = 1)
dfHS_Religion = pd.concat([X, y['HS_Religion']], axis = 1)
dfHS_Race = pd.concat([X, y['HS_Race']], axis = 1)
dfHS_Physical = pd.concat([X, y['HS_Physical']], axis = 1)
dfHS_Gender = pd.concat([X, y['HS_Gender']], axis = 1)
dfHS_Other = pd.concat([X, y['HS_Other']], axis = 1)
dfsexual_orientation = pd.concat([X, y['sexual_orientation']], axis = 1)
dfdisability = pd.concat([X, y['disability']], axis = 1)
dfnational_origin = pd.concat([X, y['national_origin']], axis = 1)
dfviolence = pd.concat([X, y['violence']], axis = 1)

In [ ]:
yAbusive0 = dfAbusive[dfAbusive['Abusive'] == 0 ].sample(5000)
yHS_Individual0 =dfHS_Individual[dfHS_Individual['HS_Individual'] == 0 ].sample(5000)
yHS_Group0 = dfHS_Group[dfHS_Group['HS_Group'] == 0 ].sample(5000)
yHS_Religion0 = dfHS_Religion[dfHS_Religion['HS_Religion'] == 0 ].sample(5000)
yHS_Race0 = dfHS_Race[dfHS_Race['HS_Race'] == 0 ].sample(5000)
yHS_Physical0 = dfHS_Physical[dfHS_Physical['HS_Physical'] == 0 ].sample(5000)
yHS_Gender0 = dfHS_Gender[dfHS_Gender['HS_Gender'] == 0 ].sample(5000)
yHS_Other0 = dfHS_Other[dfHS_Other['HS_Other'] == 0 ].sample(5000)
ysexual_orientation0 = dfsexual_orientation[dfsexual_orientation['sexual_orientation'] == 0 ].sample(5000)
ydisability0 = dfdisability[dfdisability['disability'] == 0 ].sample(5000)
ynational_origin0 =dfnational_origin[dfnational_origin['national_origin'] == 0 ].sample(5000)
yviolence0 = dfviolence[dfviolence['violence'] == 0 ].sample(5000)

In [ ]:
yAbusive1 = dfAbusive[dfAbusive['Abusive'] == 1 ]
yHS_Individual1 =dfHS_Individual[dfHS_Individual['HS_Individual'] == 1 ]
yHS_Group1 = dfHS_Group[dfHS_Group['HS_Group'] == 1 ]
yHS_Religion1 = dfHS_Religion[dfHS_Religion['HS_Religion'] == 1 ]
yHS_Race1 = dfHS_Race[dfHS_Race['HS_Race'] == 1 ]
yHS_Physical1 = dfHS_Physical[dfHS_Physical['HS_Physical'] == 1 ]
yHS_Gender1 = dfHS_Gender[dfHS_Gender['HS_Gender'] == 1 ]
yHS_Other1 = dfHS_Other[dfHS_Other['HS_Other'] == 1 ]
ysexual_orientation1 = dfsexual_orientation[dfsexual_orientation['sexual_orientation'] == 1 ]
ydisability1 = dfdisability[dfdisability['disability'] == 1 ]
ynational_origin1 =dfnational_origin[dfnational_origin['national_origin'] == 1 ]
yviolence1 = dfviolence[dfviolence['violence'] == 1 ]

In [ ]:
dfNewAbusive = pd.concat([yAbusive0, yAbusive1], axis = 0)
dfNewHS_Individual = pd.concat([yHS_Individual0, yHS_Individual1], axis = 0)
dfNewHS_Group = pd.concat([yHS_Group0, yHS_Group1], axis = 0)
dfNewHS_Religion = pd.concat([yHS_Religion0, yHS_Religion1], axis = 0)
dfNewHS_Race = pd.concat([yHS_Race0, yHS_Race1], axis = 0)
dfNewHS_Physical = pd.concat([yHS_Physical0, yHS_Physical1], axis = 0)
dfNewHS_Gender = pd.concat([yHS_Gender0, yHS_Gender1], axis = 0)
dfNewHS_Other = pd.concat([yHS_Other0, yHS_Other1], axis = 0)
dfNewsexual_orientation = pd.concat([ysexual_orientation0, ysexual_orientation1], axis = 0)
dfNewdisability = pd.concat([ydisability0, ydisability1], axis = 0)
dfNewnational_origin = pd.concat([ynational_origin0, ynational_origin1], axis = 0)
dfNewviolence = pd.concat([yviolence0, yviolence1], axis = 0)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()

In [ ]:
XAbusive, yAbusive = sm.fit_resample(dfNewAbusive.iloc[:,:-1], dfNewAbusive.iloc[:,-1])
XHS_Individual, yHS_Individual = sm.fit_resample(dfNewHS_Individual.iloc[:,:-1], dfNewHS_Individual.iloc[:,-1])
XHS_Group, yHS_Group = sm.fit_resample(dfNewHS_Group.iloc[:,:-1], dfNewHS_Group.iloc[:,-1])
XHS_Religion, yHS_Religion = sm.fit_resample(dfNewHS_Religion.iloc[:,:-1], dfNewHS_Religion.iloc[:,-1])
XHS_Race, yHS_Race = sm.fit_resample(dfNewHS_Race.iloc[:,:-1], dfNewHS_Race.iloc[:,-1])
XHS_Physical, yHS_Physical = sm.fit_resample(dfNewHS_Physical.iloc[:,:-1], dfNewHS_Physical.iloc[:,-1])
XHS_Gender, yHS_Gender = sm.fit_resample(dfNewHS_Gender.iloc[:,:-1], dfNewHS_Gender.iloc[:,-1])
XHS_Other, yHS_Other = sm.fit_resample(dfNewHS_Other.iloc[:,:-1], dfNewHS_Other.iloc[:,-1])
Xsexual_orientation, ysexual_orientation = sm.fit_resample(dfNewsexual_orientation.iloc[:,:-1], dfNewsexual_orientation.iloc[:,-1])
Xdisability, ydisability = sm.fit_resample(dfNewdisability.iloc[:,:-1], dfNewdisability.iloc[:,-1])
Xnational_origin, ynational_origin = sm.fit_resample(dfNewnational_origin.iloc[:,:-1], dfNewnational_origin.iloc[:,-1])
Xviolence, yviolence  = sm.fit_resample(dfNewviolence.iloc[:,:-1], dfNewviolence.iloc[:,-1])

KeyboardInterrupt: ignored

In [ ]:
dfAbusive = pd.concat([XAbusive, yAbusive ], axis = 1)
dfHS_Individual = pd.concat([XHS_Individual, yHS_Individual], axis = 1)
dfHS_Group = pd.concat([XHS_Group, yHS_Group], axis = 1)
dfHS_Religion = pd.concat([XHS_Religion, yHS_Religion], axis = 1)
dfHS_Race = pd.concat([XHS_Race, yHS_Race], axis = 1)
dfHS_Physical = pd.concat([XHS_Physical, yHS_Physical], axis = 1)
dfHS_Gender = pd.concat([XHS_Gender, yHS_Gender], axis = 1)
dfHS_Other = pd.concat([XHS_Other, yHS_Other], axis = 1)
dfsexual_orientation = pd.concat([Xsexual_orientation, ysexual_orientation], axis = 1)
dfdisability = pd.concat([Xdisability, ydisability], axis = 1)
dfnational_origin = pd.concat([Xnational_origin, ynational_origin], axis = 1)
dfviolence = pd.concat([Xviolence, yviolence], axis = 1)

In [ ]:
# dfAbusive.to_csv("dfAbusive.csv")
# dfHS_Individual.to_csv("dfHS_Individual.csv")
# dfHS_Group.to_csv("dfHS_Group.csv")
# dfHS_Religion.to_csv("dfHS_Religion.csv")
# dfHS_Race.to_csv("dfHS_Race.csv")
# dfHS_Physical.to_csv("dfHS_Physical.csv")
# dfHS_Gender.to_csv("dfHS_Gender.csv")
# dfHS_Other.to_csv("dfHS_Other.csv")
# dfsexual_orientation.to_csv("dfsexual_orientation.csv")
# dfdisability.to_csv("dfdisability.csv")
# dfnational_origin.to_csv("dfnational_origin.csv")
# dfviolence.to_csv("dfviolence.csv")

In [ ]:
dfAbusive = pd.read_csv(f"{DataPath}dfCatagory/dfAbusive.csv")
dfHS_Individual = pd.read_csv(f"{DataPath}dfCatagory/dfHS_Individual.csv")
dfHS_Group = pd.read_csv(f"{DataPath}dfCatagory/dfHS_Group.csv")
dfHS_Religion = pd.read_csv(f"{DataPath}dfCatagory/dfHS_Religion.csv")
dfHS_Race = pd.read_csv(f"{DataPath}dfCatagory/dfHS_Race.csv")
dfHS_Physical = pd.read_csv(f"{DataPath}dfCatagory/dfHS_Physical.csv")
dfHS_Gender = pd.read_csv(f"{DataPath}dfCatagory/dfHS_Gender.csv")
dfHS_Other = pd.read_csv(f"{DataPath}dfCatagory/dfHS_Other.csv")
dfsexual_orientation = pd.read_csv(f"{DataPath}dfCatagory/dfsexual_orientation.csv")
dfdisability = pd.read_csv(f"{DataPath}dfCatagory/dfdisability.csv")
dfnational_origin = pd.read_csv(f"{DataPath}dfCatagory/dfnational_origin.csv")
dfviolence  = pd.read_csv(f"{DataPath}dfCatagory/dfviolence.csv")


In [ ]:
dfnew = pd.concat([dfAbusive,dfHS_Individual,dfHS_Group,dfHS_Religion,dfHS_Race,dfHS_Physical,dfHS_Gender,dfHS_Other,dfsexual_orientation,dfdisability,dfnational_origin,dfviolence], axis = 0)

In [ ]:
# dfnew.fillna(np.random.randint(0,2), inplace=True)

In [ ]:
dfnew = dfnew.applymap(lambda l: l if not np.isnan(l) else np.random.choice([0, 1]))

In [ ]:
dfnew.to_csv("dfnew.csv")

In [ ]:
dfnew  = pd.read_csv(f"{DataPath}dfCatagory/dfnew3.csv")


In [ ]:
dfnew.Abusive.value_counts()

0.0    60175
1.0    59825
Name: Abusive, dtype: int64

imbalance

In [ ]:
y.columns

Index(['Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race',
       'HS_Physical', 'HS_Gender', 'HS_Other', 'sexual_orientation',
       'disability', 'national_origin', 'violence'],
      dtype='object')

In [ ]:
dfnew.HS_Individual.value_counts()

0.0    60025
1.0    59975
Name: HS_Individual, dtype: int64

In [ ]:
# yAbusive0 = y[y['Abusive'] == 0 ].Abusive.sample(5000)
# yHS_Individual0 = y[y['HS_Individual'] == 0 ].HS_Individual.sample(5000)
# yHS_Group0 = y[y['HS_Group'] == 0 ].HS_Group.sample(5000)
# yHS_Religion0 = y[y['HS_Religion'] == 0 ].HS_Religion.sample(5000)
# yHS_Race0 = y[y['HS_Race'] == 0 ].HS_Race.sample(5000)
# yHS_Physical0 = y[y['HS_Physical'] == 0 ].HS_Physical.sample(5000)
# yHS_Gender0 = y[y['HS_Gender'] == 0 ].HS_Gender.sample(5000)
# yHS_Other0 = y[y['HS_Other'] == 0 ].HS_Other.sample(5000)
# ysexual_orientation0 = y[y['sexual_orientation'] == 0 ].sexual_orientation.sample(5000)
# ydisability0 = y[y['disability'] == 0 ].disability.sample(5000)
# ynational_origin0 = y[y['national_origin'] == 0 ].national_origin.sample(5000)
# yviolence0 = y[y['violence'] == 0 ].violence.sample(5000)

In [ ]:
# yAbusive1 = y[y['Abusive'] == 1 ].Abusive
# yHS_Individual1 = y[y['HS_Individual'] == 1 ].HS_Individual
# yHS_Group1 = y[y['HS_Group'] == 1 ].HS_Group
# yHS_Religion1 = y[y['HS_Religion'] == 1 ].HS_Religion
# yHS_Race1 = y[y['HS_Race'] == 1 ].HS_Race
# yHS_Physical1 = y[y['HS_Physical'] == 1 ].HS_Physical
# yHS_Gender1 = y[y['HS_Gender'] == 1 ].HS_Gender
# yHS_Other1 = y[y['HS_Other'] == 1 ].HS_Other
# ysexual_orientation1 = y[y['sexual_orientation'] == 1 ].sexual_orientation
# ydisability1 = y[y['disability'] == 1 ].disability
# ynational_origin1 = y[y['national_origin'] == 1 ].national_origin
# yviolence1 = y[y['violence'] == 1 ].violence

In [ ]:
# yAbusive = pd.concat([yAbusive0, yAbusive1], axis = 0)
# yHS_Individual = pd.concat([yHS_Individual0, yHS_Individual1], axis = 0)
# yHS_Group = pd.concat([yHS_Group0, yHS_Group1], axis = 0)
# yHS_Religion = pd.concat([yHS_Religion0, yHS_Religion1], axis = 0)
# yHS_Race = pd.concat([yHS_Race0, yHS_Race1], axis = 0)
# yHS_Physical = pd.concat([yHS_Physical0, yHS_Physical1], axis = 0)
# yHS_Gender = pd.concat([yHS_Gender0, yHS_Gender1], axis = 0)
# yHS_Other = pd.concat([yHS_Other0, yHS_Other1], axis = 0)
# ysexual_orientation = pd.concat([ysexual_orientation0, ysexual_orientation1], axis = 0)
# ydisability = pd.concat([ydisability0, ydisability1], axis = 0)
# ynational_origin = pd.concat([ynational_origin0, ynational_origin1], axis = 0)
# yviolence = pd.concat([yviolence0, yviolence1], axis = 0)

In [ ]:
# print(yAbusive.value_counts())
# print(yHS_Individual.value_counts())
# print(yHS_Group.value_counts())
# print(yHS_Religion.value_counts())
# print(yHS_Race.value_counts())
# print(yHS_Physical.value_counts())
# print(yHS_Gender.value_counts())
# print(yHS_Other.value_counts())
# print(ysexual_orientation.value_counts())
# print(ydisability.value_counts())
# print(ynational_origin.value_counts())
# print(yviolence.value_counts())

In [ ]:
# y['Abusive'].value_counts()

In [ ]:
# y['HS_Gender'].value_counts()

In [ ]:
# y['violence'].value_counts()

In [ ]:
# yAbusive, yAbusive = sm.fit_resample(X, yAbusive)
# yHS_Individual, yHS_Individual = sm.fit_resample(X, yHS_Individual)
# yHS_Group, yHS_Group = sm.fit_resample(X, yHS_Group)
# yHS_Religion, yHS_Religion = sm.fit_resample(X, yHS_Religion)
# yHS_Race, yHS_Race = sm.fit_resample(X, yHS_Race)
# yHS_Physical, yHS_Physical = sm.fit_resample(X, yHS_Physical)
# yHS_Gender, yHS_Gender = sm.fit_resample(X, yHS_Gender)
# yHS_Other, yHS_Other = sm.fit_resample(X, yHS_Other)
# ysexual_orientation, ysexual_orientation = sm.fit_resample(X, ysexual_orientation)
# ydisability, ydisability = sm.fit_resample(X, ydisability)
# ynational_origin, ynational_origin = sm.fit_resample(X, ynational_origin)
# yviolence, yviolence  = sm.fit_resample(X, yviolence)

In [ ]:
# XHS_Individual, yHS_Individual = sm.fit_resample(X, y['HS_Individual'])

In [ ]:
# dfAbusive = pd.concat([XAbusive, yAbusive ], axis = 1)

In [ ]:
# dfAbusive['Abusive'].value_counts()

In [ ]:
# dfHS_Individual = pd.concat([XHS_Individual, yHS_Individual], axis = 1)

In [ ]:
# dfHS_Individual['HS_Individual'].value_counts()

In [ ]:
# y.Abusive.value_counts()[0]

In [ ]:
# y.columns

In [ ]:
# df0_1 = df[df.Abusive == 0].sample(8465, replace = True)
# df0_2 = df[df.HS_Individual == 0].sample(8465, replace = True)
# df0_3 = df[df.HS_Group == 0].sample(8464, replace = True)
# df0_4 = df[df.HS_Religion == 0].sample(8464, replace = True)
# df0_5 = df[df.HS_Race == 0].sample(8464, replace = True)
# df0_6 = df[df.HS_Physical == 0].sample(8464, replace = True)
# df0_7 = df[df.HS_Gender == 0].sample(8464, replace = True)
# df0_8 = df[df.HS_Other == 0].sample(8464, replace = True)
# df0_9 = df[df.sexual_orientation == 0].sample(8464, replace = True)
# df0_10 = df[df.disability == 0].sample(8464, replace = True)
# df0_11 = df[df.national_origin == 0].sample(8464, replace = True)
# df0_12 = df[df.violence == 0].sample(8464, replace = True)

In [ ]:
# df_new0 = pd.concat([df0_1, df0_2, df0_3, df0_4, df0_5, df0_6, df0_7, df0_8, df0_9, df0_10, df0_11, df0_12], axis = 0)

In [ ]:
# df_new0.shape

In [ ]:
# df1_1 = df[df.Abusive == 1].sample(8465, replace = True)
# df1_2 = df[df.HS_Individual == 1].sample(8465, replace = True)
# df1_3 = df[df.HS_Group == 1].sample(8464, replace = True)
# df1_4 = df[df.HS_Religion == 1].sample(8464, replace = True)
# df1_5 = df[df.HS_Race == 1].sample(8464, replace = True)
# df1_6 = df[df.HS_Physical == 1].sample(8464, replace = True)
# df1_7 = df[df.HS_Gender == 1].sample(8464, replace = True)
# df1_8 = df[df.HS_Other == 1].sample(8464, replace = True)
# df1_9 = df[df.sexual_orientation == 1].sample(8464, replace = True)
# df1_10 = df[df.disability == 1].sample(8464, replace = True)
# df1_11 = df[df.national_origin == 1].sample(8464, replace = True)
# df1_12 = df[df.violence == 1].sample(8464, replace = True)

In [ ]:
# df_new1 = pd.concat([df1_1, df1_2, df1_3, df1_4, df1_5, df1_6, df1_7, df1_8, df1_9, df1_10, df1_11, df1_12], axis = 0)

In [ ]:
# df_new1.shape

In [ ]:
# df_new = pd.concat([df_new0, df_new1], axis = 0)

In [ ]:
# df_new.shape

In [ ]:
# dfAbusive1 = df[df.Abusive == 1]
# dfAbusive0 = df[df.Abusive == 0]
# dfHS_Individual1 = df[df.HS_Individual == 1]
# dfHS_Individual0 = df[df.HS_Individual == 0]
# dfHS_Group1 = df[df.HS_Group == 1]
# dfHS_Group0 = df[df.HS_Group == 0]
# dfHS_Religion1 = df[df.HS_Religion == 1]
# dfHS_Religion0 = df[df.HS_Religion == 0]
# dfHS_Race1 = df[df.HS_Race == 1]
# dfHS_Race0 = df[df.HS_Race == 0]
# dfHS_Physical1 = df[df.HS_Physical == 1]
# dfHS_Physical0 = df[df.HS_Physical == 0]
# dfHS_Gender1 = df[df.HS_Gender == 1]
# dfHS_Gender0 = df[df.HS_Gender == 0]
# dfHS_Other1 = df[df.HS_Other == 1]
# dfHS_Other0 = df[df.HS_Other == 0]
# dfsexual_orientation1 = df[df.sexual_orientation == 1]
# dfsexual_orientation0 = df[df.sexual_orientation == 0]
# dfdisability1 = df[df.disability == 1]
# dfdisability0 = df[df.disability == 0]
# dfnational_origin1 = df[df.national_origin == 1]
# dfnational_origin0 = df[df.national_origin == 0]
# dfviolence1 = df[df.violence == 1]
# dfviolence0 = df[df.violence == 0]


In [ ]:
# dfAbusive0 = dfAbusive0.sample(y.Abusive.value_counts()[0], replace= True)
# dfAbusive1 = dfAbusive1.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Individual1 = dfHS_Individual1.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Individual0 = dfHS_Individual0.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Group1 = dfHS_Group1.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Group0 = dfHS_Group0.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Religion1 = dfHS_Religion1.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Religion0 = dfHS_Religion0.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Race1 = dfHS_Race1.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Race0 = dfHS_Race0.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Physical1 = dfHS_Physical1.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Physical0 = dfHS_Physical0.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Gender1 = dfHS_Gender1.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Gender0 = dfHS_Gender0.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Other1 = dfHS_Other1.sample(y.Abusive.value_counts()[0], replace= True)
# dfHS_Other0 = dfHS_Other0.sample(y.Abusive.value_counts()[0], replace= True)
# dfsexual_orientation1 = dfsexual_orientation1.sample(y.Abusive.value_counts()[0], replace= True)
# dfsexual_orientation0 = dfsexual_orientation0.sample(y.Abusive.value_counts()[0], replace= True)
# dfdisability1 = dfdisability1.sample(y.Abusive.value_counts()[0], replace= True)
# dfdisability0 = dfdisability0.sample(y.Abusive.value_counts()[0], replace= True)
# dfnational_origin1 = dfnational_origin1.sample(y.Abusive.value_counts()[0], replace= True)
# dfnational_origin0 = dfnational_origin0.sample(y.Abusive.value_counts()[0], replace= True)
# dfviolence1 = dfviolence1.sample(y.Abusive.value_counts()[0], replace= True)
# dfviolence0 = dfviolence0.sample(y.Abusive.value_counts()[0], replace= True)

In [ ]:
# dfAbusive = pd.concat([dfAbusive0, dfAbusive1], axis = 0)
# # dfAbusive0.sample(y.Abusive.value_counts()[0], replace= True)
# # dfAbusive1.sample(y.Abusive.value_counts()[0], replace= True)

# dfHS_Individual = pd.concat([dfHS_Individual0, dfHS_Individual1], axis = 0)
# # dfHS_Individual1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfHS_Individual0.sample(y.Abusive.value_counts()[0], replace= True)

# dfHS_Group = pd.concat([dfHS_Group0,dfHS_Group1], axis = 0)
# # dfHS_Group1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfHS_Group0.sample(y.Abusive.value_counts()[0], replace= True)

# dfHS_Religion = pd.concat([dfHS_Religion0, dfHS_Religion1], axis = 0)
# # dfHS_Religion1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfHS_Religion0.sample(y.Abusive.value_counts()[0], replace= True)

# dfHS_Race = pd.concat([dfHS_Race0, dfHS_Race1], axis = 0)
# # dfHS_Race1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfHS_Race0.sample(y.Abusive.value_counts()[0], replace= True)

# dfHS_Physical = pd.concat([dfHS_Physical0, dfHS_Physical1], axis = 0)
# # dfHS_Physical1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfHS_Physical0.sample(y.Abusive.value_counts()[0], replace= True)

# dfHS_Gender = pd.concat([dfHS_Gender0, dfHS_Gender1], axis = 0)
# # dfHS_Gender1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfHS_Gender0.sample(y.Abusive.value_counts()[0], replace= True)

# dfHS_Other = pd.concat([dfHS_Other0, dfHS_Other1], axis = 0)
# # dfHS_Other1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfHS_Other0.sample(y.Abusive.value_counts()[0], replace= True)

# dfsexual_orientation = pd.concat([dfsexual_orientation0, dfsexual_orientation1], axis = 0)
# # dfsexual_orientation1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfsexual_orientation0.sample(y.Abusive.value_counts()[0], replace= True)

# dfdisability = pd.concat([dfdisability0, dfdisability1], axis = 0)
# # dfdisability1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfdisability0.sample(y.Abusive.value_counts()[0], replace= True)

# dfnational_origin = pd.concat([dfnational_origin0, dfnational_origin1], axis = 0)
# # dfnational_origin1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfnational_origin0.sample(y.Abusive.value_counts()[0], replace= True)

# dfviolence = pd.concat([dfviolence0, dfviolence1], axis = 0)
# # dfviolence1.sample(y.Abusive.value_counts()[0], replace= True)
# # dfviolence0.sample(y.Abusive.value_counts()[0], replace= True)

Preprocess Input

In [ ]:
def preprocess(sentence:list):
  # case Folding
  sentences = [sentence.lower() for sentence in sentence]
  # print(sentences)


  # tokenizer
  tokenizer     = RegexpTokenizer(r'(?<!\S)[A-Za-z]+(?!\S)|(?<!\S)[A-Za-z]+(?=:(?!\S))')
  # hasil_token   = tokenizer.tokenize(sentences[1])
  token_sentence = [tokenizer.tokenize(sentence) for sentence in sentences]
  # print(token_sentence)


  # ah, al, ala, an, and, as, atau, awal, be, ber, bin, bro, bu, bung, but, by, cak, cc, dan, deh, dengan, di, dil, dong, dua, eh, fadli, fahri
  # fan, for, front, full, harto, hasan, he, hehe, hem, ia, ih, in, is, itu, iya, je, juga, kah, kak, kan, kang, kat, "kau", ke, km, kok, ku, ky
  # lah, like, lima, live, "lo", me, mei, men, min, mk, mu, my, nah, nak, namun, nek, nih, no, nya, of, oh, oi, oke, -pada, pak, per, ppp, pun, ra
  # rang, rt, sa, se, seba, si, sing, ta, tang, tar, tau, the, ting, to, toh, tomohon, uniform, up, untuk, user, wah, wkwk, ya, yaitu, yakni, yang, zon

  # yuk,yg,yaa,wkwkwk,tdk,so,kyk,ko,jg,jgn,jd,jdi,hrs,haha,gw,gk,ga,ente,emg,elo,dr,dlm,dgn,byk,bkn,aq,ama,aku,adlh,ad

  # ad,adlh,aku,ama,aq,bgt,bkn,blg,bnyk,bs,byk,da,dah,dapet,dapat,dengar,dg,dgn,dll,ga,gmn,gt,gw,haha,hahaha,jd,jdi,jgn,ka,karna,ken,kl,klo,kyk,lg,lho,lu,mmg,mo,msh,ngga,nggak,ntar,yg,yo
  words = """ad,adlh,aku,ama,aq,bgt,bkn,blg,bnyk,bs,byk,da,dah,dapet,dapat,dengar,dg,dgn,dll,ga,gmn,gt,gw,haha,hahaha,jd,jdi,jgn,ka,karna,ken,kl,klo,kyk,lg,lho,lu,mmg,mo,msh,ngga,nggak,ntar,yg,yo,yuk,yg,yaa,wkwkwk,tdk,so,kyk,ko,jg,jgn,jd,jdi,hrs,haha,gw,gk,ga,ente,emg,elo,dr,dlm,dgn,byk,bkn,aq,ama,aku,adlh,ad,gue,aja,aku,pengguna,ah,al,ala,an,and,as,atau,awal,be,ber,bin,bro,bu,bung,but,by,cak,cc,dan,deh,dengan,di,dil,dong,dua,eh,fadli,fahri,fan,for,front,full,harto,hasan,he,hehe,hem,ia,ih,in,is,itu,iya,je,juga,kah,kak,kan,kang,kat,kau,ke,km,kok,ku,ky,lah,like,lima,live,lo,me,mei,men,min,mk,mu,my,nah,nak,namun,nek,nih,no,nya,of,oh,oi,oke,pak,per,ppp,pun,ra,rang,rt,sa,se,seba,si,sing,ta,tang,tar,ter,tau,the,ting,to,toh,tomohon,uniform,up,untuk,user,wah,wkwk,ya,yaitu,yakni,yang,zon"""
  word = words.split(",")
  # word = np.reshape(word, -1)
  # word.tolist()
  # stopword
  wordlist= set(stopwords.words('indonesian') + word)
  sw_sentences = [[word for word in sentence if word not in wordlist]for sentence in token_sentence]
  cleaned_sentences = [' '.join(sentence) for sentence in sw_sentences]
  # print(cleaned_sentences)


  #[str(sentence) for sentence in sw_sentences]
  # stemming
  stemmer     = StemmerFactory().create_stemmer()
  # input_stemm   = str(sw_sentences)
  # hasil_stemm   = stemmer.stem(input_stemm)
  # print (hasil_stemm)
  stem_sentence = [stemmer.stem(sentence) for sentence in cleaned_sentences]
  # print(stem_sentence)
  return stem_sentence

In [ ]:
print(preprocess(["Mereka harus dibunuh secepat mungkin"]))

['bunuh cepat']


Klasifikasi

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def confusion(y_test, y_predict, df_target):
  jumlah_akurasi = 0
  for i in range(len(y.columns)):
    print("\n",df_target.columns[i])
    cm_y1_forest = confusion_matrix(y_test[:,i], y_predict[:,i])
    print(cm_y1_forest)

    jumlah_akurasi += accuracy_score(y_test[:,i], y_predict[:,i])
    rata2 = jumlah_akurasi/(len(df_target.columns))
    # print("akurasi: %.2f"%accuracy_score(y_test[:,i], y_predict[:,i]))

    cm_y1_forest = classification_report(y_test[:,i], y_predict[:,i])
    print(cm_y1_forest)
    print("->"*50)
  print("akuras rata-rata : {:.2f}%".format(rata2))


In [ ]:
dfnew

,Unnamed: 0.1,Unnamed: 0,abang,abis,abu,acara,ada,adab,adat,ade,...,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,sexual_orientation,disability,national_origin,violence
0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
3,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
4,4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,9995,9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
119996,9996,9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
119997,9997,9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
119998,9998,9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
dfnew.drop(columns = "Unnamed: 0", inplace = True)

In [ ]:
dfnew.reset_index(inplace=True)

In [ ]:
dfnew.drop(columns = "index", inplace = True)

In [ ]:
dfnew.iloc[:,-12:].head()

,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,sexual_orientation,disability,national_origin,violence
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
4,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0


In [ ]:
dfnew.drop(columns = "Unnamed: 0.1", inplace=True)

In [ ]:
X = dfnew.iloc[:,:-12]
y = dfnew.iloc[:,-12:]

In [ ]:
y.head()

,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,sexual_orientation,disability,national_origin,violence
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
4,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0


In [ ]:
X.head()

,abang,abis,abu,acara,ada,adab,adat,ade,adek,adem,...,yah,yahudi,yakin,yaudah,ye,you,zalim,zaman,zionis,zonk
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dfnew.HS_Individual.value_counts()

0.0    60025
1.0    59975
Name: HS_Individual, dtype: int64

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DTree = MultiOutputClassifier(DecisionTreeClassifier())

In [ ]:
DTree.fit(x_train.values,y_train.values)

MultiOutputClassifier(estimator=DecisionTreeClassifier())

In [ ]:
y_predic = DTree.predict(x_test.values)

In [ ]:
confusion(y_test=y_test.values, y_predict=y_predic, df_target=y)


 Abusive
[[6570 5569]
 [6364 5497]]
              precision    recall  f1-score   support

         0.0       0.51      0.54      0.52     12139
         1.0       0.50      0.46      0.48     11861

    accuracy                           0.50     24000
   macro avg       0.50      0.50      0.50     24000
weighted avg       0.50      0.50      0.50     24000

->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->

 HS_Individual
[[7028 5036]
 [6826 5110]]
              precision    recall  f1-score   support

         0.0       0.51      0.58      0.54     12064
         1.0       0.50      0.43      0.46     11936

    accuracy                           0.51     24000
   macro avg       0.51      0.51      0.50     24000
weighted avg       0.51      0.51      0.50     24000

->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->

 HS_Group
[[7027 5012]
 [6797 5164]]
              precision   

In [ ]:
from sklearn.tree import ExtraTreeClassifier

In [ ]:
EXTree = MultiOutputClassifier(ExtraTreeClassifier())

In [ ]:
EXTree.fit(x_train.values,y_train.values)

MultiOutputClassifier(estimator=ExtraTreeClassifier())

In [ ]:
y_predic = EXTree.predict(x_test.values)

confusion(y_test=y_test.values, y_predict=y_predic, df_target=y)


 Abusive
[[6546 5593]
 [6382 5479]]
              precision    recall  f1-score   support

         0.0       0.51      0.54      0.52     12139
         1.0       0.49      0.46      0.48     11861

    accuracy                           0.50     24000
   macro avg       0.50      0.50      0.50     24000
weighted avg       0.50      0.50      0.50     24000

->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->

 HS_Individual
[[6995 5069]
 [6829 5107]]
              precision    recall  f1-score   support

         0.0       0.51      0.58      0.54     12064
         1.0       0.50      0.43      0.46     11936

    accuracy                           0.50     24000
   macro avg       0.50      0.50      0.50     24000
weighted avg       0.50      0.50      0.50     24000

->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->

 HS_Group
[[7064 4975]
 [6843 5118]]
              precision   

In [ ]:
# from sklearn.ensemble import ExtraTreesClassifier #kelamaan

In [ ]:
# EnsTree = MultiOutputClassifier(ExtraTreesClassifier())

In [ ]:
# EnsTree.fit(x_train.values, y_train.values)

In [ ]:
# y_predic = EnsTree.predict(x_test.values)

In [ ]:
#  confusion(y_test=y_test.values, y_predict=y_predic, df=df_cleaned)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
Knn = KNeighborsClassifier(n_neighbors=3)
Knn.fit(x_train.values, y_train.values)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
y_predic = Knn.predict(x_test.values)



In [ ]:
confusion(y_test=y_test.values, y_predict=y_predic, df_target=y)


 Abusive
[[6058 6081]
 [5927 5934]]
              precision    recall  f1-score   support

         0.0       0.51      0.50      0.50     12139
         1.0       0.49      0.50      0.50     11861

    accuracy                           0.50     24000
   macro avg       0.50      0.50      0.50     24000
weighted avg       0.50      0.50      0.50     24000

->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->

 HS_Individual
[[6013 6051]
 [5904 6032]]
              precision    recall  f1-score   support

         0.0       0.50      0.50      0.50     12064
         1.0       0.50      0.51      0.50     11936

    accuracy                           0.50     24000
   macro avg       0.50      0.50      0.50     24000
weighted avg       0.50      0.50      0.50     24000

->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->

 HS_Group
[[5909 6130]
 [5779 6182]]
              precision   

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RForest = MultiOutputClassifier(RandomForestClassifier())
RForest.fit(x_train.values, y_train.values)

MultiOutputClassifier(estimator=RandomForestClassifier())

In [ ]:
y_predic = RForest.predict(x_test.values)

confusion(y_test=y_test.values, y_predict=y_predic,df_targetf=y)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic = MultiOutputClassifier(LogisticRegression())

In [ ]:
logistic.fit(x_train.values,y_train.values)

In [ ]:
y_predic = logistic.predict(x_test.values)

In [ ]:
y_predic

In [ ]:
df_cleaned.columns

In [ ]:
confusion(y_test=y_test.values, y_predict=y_predic, df_target=y)